In [1]:
!pip install -U ../input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/qubvel/efficientnet-1.0.0-py3-none-any.whl
!pip install ../input/qubvel/image_classifiers-1.0.0-py3-none-any.whl
!pip install ../input/qubvel/segmentation_models-1.0.0-py3-none-any.whl

import segmentation_models as sm
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import glob
import pathlib
from tqdm.auto import tqdm
gc.enable()
import cv2
from PIL import Image
from tensorflow.keras.models import load_model
from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf

Processing /kaggle/input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/qubvel/efficientnet-1.0.0-py3-none-any.whl
Processing /kaggle/input/qubvel/image_classifiers-1.0.0-py3-none-any.whl
Processing /kaggle/input/qubvel/segmentation_models-1.0.0-py3-none-any.whl
Segmentation Models: using `keras` framework.


In [2]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)

In [3]:
cfg = {
    'model_path': '../input/tiled-256p-fpn-effnetb3-jaccard-loss/best_model.h5',
    'img_shape': 1024,
    'model_img_shape': 256,
    'slices': 4
}

def mask2rle(mask, orig_dim=160):
    #Rescale image to original size
    size = int(len(mask.flatten())**.5)
    n = Image.fromarray(mask.reshape((size, size))*255.0)
    n = n.resize((orig_dim, orig_dim))
    n = np.array(n).astype(np.float32)
    #Get pixels to flatten
    pixels = n.T.flatten()
    #Round the pixels using the half of the range of pixel value
    pixels = (pixels-min(pixels) > ((max(pixels)-min(pixels))/2)).astype(int)
    pixels = np.nan_to_num(pixels) #incase of zero-div-error
    
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)


def generate_tile(img, slices=cfg['slices']):
    factor = int(img.shape[0] / slices)
    img_tiles = []
    temp_img = None
    for x in range(slices):
        for y in range(slices):
            temp_img = img[factor*x : factor*(x+1), factor*y: factor*(y+1), ...]
            img_tiles.append(temp_img)
            del temp_img; gc.collect()
    return img_tiles


def tiles2img(
    tiles: list, #list of mask tiles
    display = False,
    slices=cfg['slices']
):
    '''
    Converts tiles of images and masks to one array
    '''
    img = []
    temp_img = None
    for i in range(slices):
        temp_img = tiles[slices*i: slices*(i+1)]
        temp_img = np.hstack([i for i in temp_img])
        img.append(temp_img)
        del temp_img
    mask = np.vstack([i for i in img])
    if display:
        print(mask.shape)
        plt.imshow(mask)
        plt.axis('off')
    return mask

In [4]:
model = load_model(cfg['model_path'], custom_objects={'bce_dice_loss': bce_dice_loss, 'dice_coef':dice_coef, 'iou_coef': iou_coef, 'dice_loss': dice_loss, 'binary_crossentropy_plus_jaccard_loss': sm.losses.bce_jaccard_loss, 'f1-score': sm.metrics.f1_score, 'binary_crossentropy_plus_dice_loss':sm.losses.bce_dice_loss})
#model.summary()

2022-09-10 09:03:00.102502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-10 09:03:00.223481: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-10 09:03:00.224264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-10 09:03:00.225996: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
test_df = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')
sub = {'id':[], 'rle':[]}

for _, row in tqdm(test_df.iterrows()):
    path = '../input/hubmap-organ-segmentation/test_images/' + str(row['id']) + '.tiff'
    img = Image.open(path)
    img = img.resize((cfg['img_shape'], cfg['img_shape']))
    img = np.array(img); img = img / 255.0
    img_tiles = generate_tile(img); del img
    mask_tiles = []
    for img in img_tiles:
        img = img.reshape(1, cfg['model_img_shape'], cfg['model_img_shape'], 3)
        mask = model.predict(img)
        mask_tiles.append(mask)
    mask = tiles2img(mask_tiles)
    rle = mask2rle(np.round(mask).astype(np.float32), row['img_width']) 
    sub['id'].append(row['id'])
    sub['rle'].append(rle)
    
    
sub = pd.DataFrame(sub)
sub.to_csv('submission.csv', index=False)
sub

0it [00:00, ?it/s]

2022-09-10 09:03:13.106651: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-10 09:03:15.918521: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


,id,rle
0,10078,1150 27 3173 27 5198 25 7221 25 9240 29 11263 ...
